## Setup

In [ ]:
# Setup
import nltk 
nltk.download('framenet_v17')

In [ ]:
# Now we need nlp for real.
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [ ]:
#%store nlp

%store -r  nlp

In [2]:
from app.classes.operations.dependencies import Dependencies
from app.src.user_scripts.main_ops.user_deps import get_dependencies

inner_deps = Dependencies(nlp=None, fake=True)

demo_deps = get_dependencies(inner_deps)

## Contract

In [3]:
# Get the template
from app.templates.template_getter import get_template
contract = get_template('sample_t')
#contract.print_all_strings()

In [ ]:
#print(contract.to_sym())
#print(contract.nl_template.stringify())

In [4]:
# User chooses the operation
ops_dict = {
    1: 'Customize a Parameter',
    2: 'Add a termination condition',
    #3: 'Add a new domain event'
}

for opi in ops_dict:
    print(f'{opi} - {ops_dict[opi]}')
    
selected_op = int(input('Select an operation: '))

print(f'\nSelected operation: {ops_dict[selected_op]}\n')

1 - Customize a Parameter
2 - Add a termination condition

Selected operation: Customize a Parameter



In [5]:
from app.src.user_scripts.main_ops.refine_parm import refine_parm
from app.src.user_scripts.main_ops.add_term import add_term
#from app.src.user_scripts.main_ops.add_dm import add_dm

if selected_op == 1:
    refine_parm(demo_deps, contract)

elif selected_op == 2:
    add_term(demo_deps, contract)

# elif selected_op == 3: 
#     add_dm(deps, contract)

ImportError: cannot import name 'IExtractNounPhrase' from 'app.src.extractors.custom_event.noun_phrase_extractor' (c:\Users\regan\Documents\dev\symboleo-nlp\app\src\extractors\custom_event\noun_phrase_extractor.py)

In [ ]:
#print(contract.to_sym())
contract.print_all_strings()

In [ ]:
class TT:
    f = 4